# Pytorch基础
## 层和块
多层感知机的定义

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) # 实现一个单隐藏层的多层感知机
''' Sequetial 定义一个module'''

X = torch.rand(2, 20)
net(X)

tensor([[-0.0854, -0.1793, -0.0371, -0.1509,  0.1337,  0.1839, -0.1542,  0.1185,
         -0.0266, -0.0417],
        [-0.0392, -0.1843, -0.0459,  0.0202,  0.0970,  0.1986, -0.2344,  0.0579,
         -0.1142, -0.0763]], grad_fn=<AddmmBackward0>)

`注意`：另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。 这实际上是net.__call__(X)的简写。 这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

### 自定义块

In [7]:
class MLP(nn.Module): # 继承nn.Module
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__() # Module的内部定义函数
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后再每次调用正向传播函数的时候调用这些层。

In [8]:
net = MLP()
net(X)

tensor([[ 0.1537,  0.0390,  0.0097,  0.1208, -0.1288, -0.0551,  0.0346, -0.0111,
          0.0511, -0.0720],
        [ 0.1985,  0.0145,  0.1463,  0.1481, -0.1782, -0.0382, -0.0850,  0.1092,
          0.1398, -0.1581]], grad_fn=<AddmmBackward0>)

### 顺序块

In [11]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict 相当于把输入的块按照顺序进行前向传播
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

实例化

In [12]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1231,  0.1269, -0.1406,  0.1004,  0.1917,  0.2669,  0.2969,  0.0465,
         -0.1975, -0.1368],
        [ 0.3482,  0.2575, -0.2779,  0.2274,  0.0530,  0.3030,  0.2096,  0.1612,
         -0.1653, -0.1826]], grad_fn=<AddmmBackward0>)

### 在正向传播函数中执行代码

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0669, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1481, grad_fn=<SumBackward0>)

## 参数管理
我们首先关注具有单隐藏层的多层感知机

In [19]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.7239],
        [0.6009]], grad_fn=<AddmmBackward0>)

### 访问参数
拿到第二个全连接层的参数

In [20]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[0.2921, 0.2820, 0.1808, 0.2005, 0.1276, 0.2706, 0.1245, 0.0384]])), ('bias', tensor([0.1194]))])


### 访问目标参数

In [24]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data) # 访问真正值
net[2].weight.grad == None # 访问参数的梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1194], requires_grad=True)
tensor([0.1194])


True

### 一次性访问所有参数

In [25]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [27]:
net.state_dict()['2.bias'].data

tensor([0.1194])

### 从嵌套块中收集参数

In [29]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1()) # 传字符串名字进去 类似字典
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3523],
        [0.3523]], grad_fn=<AddmmBackward0>)

看看内部的结构组织

In [30]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 内置初始化

In [35]:
def init_normal(m): # 正态分布
    if type(m) == nn.Linear: # 线性全连接层
        nn.init.normal_(m.weight, mean=0, std=0.01) # 替换函数
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0091,  0.0068, -0.0047,  0.0150]), tensor(0.))

初始化一个常数

In [36]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对不同块进行不同的参数初始化方法

In [37]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6349, -0.4538,  0.6115, -0.6082])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

In [38]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[6.6218, 7.7870, -0.0000, 0.0000],
        [-0.0000, 8.7488, 0.0000, 0.0000]], grad_fn=<SliceBackward0>)

### 直接做数值替换

In [39]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.7870,  1.0000,  1.0000])

### 参数绑定

In [42]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1)) # 第二个和第三个隐藏层的参数share 是一样的
net(X)
# 检查参数是否相同（ReLu没有参数）
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层
层也是`nn.Module`的子类。首先构造一个没有任何参数的自定义层。

In [44]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将自定义层放入更复杂的模型之中

In [45]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

### 带参数的图层

In [51]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 自定义参数要放入self.weight中使用Parameter类
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.3802, -2.4837,  1.3387],
        [-1.1256,  0.2989,  1.3690],
        [-2.4984, -1.1459,  0.9512],
        [-1.2212,  0.6478,  0.0128],
        [ 0.3398, -0.6140, -0.3520]], requires_grad=True)

使用自定义层直接执行正向传播计算

In [54]:
dense(torch.rand(2,5))

tensor([[0.0000, 0.0000, 0.4251],
        [0.0000, 0.0000, 0.3567]])

使用自定义层构建模型

In [55]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[13.6480],
        [ 1.3567]])

## 读写文件

In [56]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后将它们读回内存

In [57]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [59]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [60]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

将模型的参数存储一个叫做`mlp.params`的文件

In [61]:
torch.save(net.state_dict(), 'mlp.params')

实现文件读取参数，需要预先存储一次多层感知机器的备份。直接读取文件中存储的参数。

In [62]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [63]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])